In [1]:
import pandas as pd
import os

metadata_path = '/Users/siddhantaggarwal/Desktop/Battery_RUL/cleaned_dataset/metadata.csv' # Make sure this path is correct
metadata = pd.read_csv(metadata_path)
print("Original metadata head:")
print(metadata.head())
print("\nMetadata info:")
metadata.info()

Original metadata head:
        type                                         start_time  \
0  discharge  [2010.       7.      21.      15.       0.    ...   
1  impedance  [2010.       7.      21.      16.      53.    ...   
2     charge  [2010.       7.      21.      17.      25.    ...   
3  impedance                    [2010    7   21   20   31    5]   
4  discharge  [2.0100e+03 7.0000e+00 2.1000e+01 2.1000e+01 2...   

   ambient_temperature battery_id  test_id  uid   filename  \
0                    4      B0047        0    1  00001.csv   
1                   24      B0047        1    2  00002.csv   
2                    4      B0047        2    3  00003.csv   
3                   24      B0047        3    4  00004.csv   
4                    4      B0047        4    5  00005.csv   

             Capacity                   Re                  Rct  
0  1.6743047446975208                  NaN                  NaN  
1                 NaN  0.05605783343888099  0.20097016584458333  
2 

In [2]:
discharge_metadata_df = metadata[metadata['type'] == 'discharge'].copy()
print("\nDischarge metadata head:")
print(discharge_metadata_df.head())
print(f"\nNumber of discharge entries: {len(discharge_metadata_df)}")


Discharge metadata head:
         type                                         start_time  \
0   discharge  [2010.       7.      21.      15.       0.    ...   
4   discharge  [2.0100e+03 7.0000e+00 2.1000e+01 2.1000e+01 2...   
6   discharge  [2.010e+03 7.000e+00 2.200e+01 1.000e+00 4.000...   
8   discharge  [2010.       7.      22.       6.      16.    ...   
10  discharge  [2010.       7.      22.      10.      51.    ...   

    ambient_temperature battery_id  test_id  uid   filename  \
0                     4      B0047        0    1  00001.csv   
4                     4      B0047        4    5  00005.csv   
6                     4      B0047        6    7  00007.csv   
8                     4      B0047        8    9  00009.csv   
10                    4      B0047       10   11  00011.csv   

              Capacity   Re  Rct  
0   1.6743047446975208  NaN  NaN  
4   1.5243662105099023  NaN  NaN  
6   1.5080762969973425  NaN  NaN  
8   1.4835577960067696  NaN  NaN  
10  1.46713

In [3]:
from pathlib import Path
import pandas as pd

root = Path("/Users/siddhantaggarwal/Desktop/Battery_RUL/cleaned_dataset")
meta = pd.read_csv(root / "metadata.csv")

# Coerce numeric columns
for col in ["Capacity", "Re", "Rct"]:
    meta[col] = pd.to_numeric(meta[col], errors="coerce")

# Filter cleanly (avoid query to sidestep dtype issues)
dis = meta.loc[
    (meta["type"] == "discharge") &
    (meta["Capacity"].notna()) &
    (meta["Capacity"] > 0)
].copy()

# Cycle index per battery
dis["cycle_index"] = dis.groupby("battery_id").cumcount() + 1

# SOH and EOL/RUL
cap0 = dis.groupby("battery_id")["Capacity"].transform("first")
dis["SOH"] = dis["Capacity"] / cap0

eol_cycle = (dis.loc[dis["SOH"] <= 0.8]
               .groupby("battery_id")["cycle_index"]
               .min()
               .rename("EOL_cycle"))
dis = dis.merge(eol_cycle, on="battery_id", how="left")
dis["RUL"] = dis["EOL_cycle"] - dis["cycle_index"]

In [5]:
from pathlib import Path
import pandas as pd
import numpy as np

# Paths
root = Path("/Users/siddhantaggarwal/Desktop/Battery_RUL/cleaned_dataset")
processed_dir = Path("/Users/siddhantaggarwal/Desktop/Battery_RUL/data/processed")
processed_dir.mkdir(parents=True, exist_ok=True)

# 1) Load metadata (assumes previous cell coerced numeric columns)
meta = pd.read_csv(root / "metadata.csv")
for col in ["Capacity", "Re", "Rct"]:
    meta[col] = pd.to_numeric(meta[col], errors="coerce")

# Discharge-only rows with valid capacity
dis = meta.loc[
    (meta["type"] == "discharge") & meta["Capacity"].notna() & (meta["Capacity"] > 0)
].copy()
dis["cycle_index"] = dis.groupby("battery_id").cumcount() + 1

# SOH, EOL, RUL
cap0 = dis.groupby("battery_id")["Capacity"].transform("first")
dis["SOH"] = dis["Capacity"] / cap0

eol_cycle = (
    dis.loc[dis["SOH"] <= 0.8]
      .groupby("battery_id")["cycle_index"]
      .min()
      .rename("EOL_cycle")
)
dis = dis.merge(eol_cycle, on="battery_id", how="left")
dis["RUL"] = dis["EOL_cycle"] - dis["cycle_index"]

# 2) Feature extraction from per-cycle waveform CSVs

def extract_cycle_features(file_path: Path) -> dict:
    try:
        if not file_path.exists():
            return {"ok": False, "filename": str(file_path.name)}
        ts = pd.read_csv(file_path)
    except Exception as e:
        return {"ok": False, "filename": str(file_path.name), "error": str(e)}

    # Ensure required columns exist
    required = {"Voltage_measured", "Current_measured", "Temperature_measured", "Time"}
    if not required.issubset(ts.columns):
        return {"ok": False, "filename": str(file_path.name)}

    # Basic stats
    try:
        v = ts["Voltage_measured"].astype(float)
        i = ts["Current_measured"].astype(float)
        t = ts["Time"].astype(float)
        temp = ts["Temperature_measured"].astype(float)
    except Exception as e:
        return {"ok": False, "filename": str(file_path.name), "error": str(e)}

    if len(t) < 2:
        return {"ok": False, "filename": str(file_path.name)}

    # Duration (s)
    duration_s = float(t.iloc[-1] - t.iloc[0])

    # Approx coulomb count during discharge (Ah): integrate |I| over time
    dt = np.diff(t.to_numpy(), prepend=t.iloc[0])
    coulomb_Asec = np.sum(np.abs(i.to_numpy()) * dt)
    coulomb_Ah = float(coulomb_Asec / 3600.0)

    # Simple IR drop proxy: voltage drop between first non-zero current and the next sample
    nz = np.where(np.abs(i.to_numpy()) > 1e-3)[0]
    if len(nz) >= 2:
        ir_drop = float(v.iloc[nz[0]] - v.iloc[nz[1]])
    else:
        ir_drop = np.nan

    # Voltage/temperature features
    feats = {
        "ok": True,
        "duration_s": duration_s,
        "voltage_mean": float(v.mean()),
        "voltage_min": float(v.min()),
        "voltage_max": float(v.max()),
        "temp_max": float(temp.max()),
        "current_mean_abs": float(np.abs(i).mean()),
        "coulomb_Ah": coulomb_Ah,
        "ir_drop_proxy": ir_drop,
    }
    return feats

# Map filenames -> features
data_dir = root / "data"
print(f"Extracting features from {len(dis['filename'].unique())} files...")
records = []
for i, fn in enumerate(dis["filename"].unique()):
    if i % 100 == 0:
        print(f"  Processed {i}/{len(dis['filename'].unique())} files...")
    fp = data_dir / fn
    feats = extract_cycle_features(fp)
    feats["filename"] = fn  # Always set filename
    records.append(feats)

print("Creating features DataFrame...")
feat_df = pd.DataFrame.from_records(records)

# Check if we have any successful extractions
if "ok" not in feat_df.columns:
    raise ValueError("No 'ok' column found in feat_df. Check extract_cycle_features function.")

# Filter to successful extractions only
successful = feat_df["ok"] == True
if successful.sum() == 0:
    raise ValueError("No successful feature extractions! Check file paths and CSV structure.")
    
feat_df = feat_df.loc[successful].copy()
feat_df = feat_df.drop(columns=["ok"])  # Remove the ok column

print(f"Successfully extracted features from {len(feat_df)}/{len(records)} files")

# 3) Join features with labels
joined = dis.merge(feat_df, on="filename", how="inner")
print(f"Joined dataset: {len(joined)} rows after merge")

# 4) Create battery-level splits (train/val/test by battery_id)
unique_batts = joined["battery_id"].unique()
if len(unique_batts) == 0:
    raise ValueError("No batteries found after joining!")

rng = np.random.default_rng(42)
rng.shuffle(unique_batts)

n = len(unique_batts)
train_batts = set(unique_batts[: int(0.7 * n)])
val_batts = set(unique_batts[int(0.7 * n) : int(0.85 * n)])
test_batts = set(unique_batts[int(0.85 * n) :])

def split_of(bid: str) -> str:
    if bid in train_batts:
        return "train"
    if bid in val_batts:
        return "val"
    return "test"

joined["split"] = joined["battery_id"].map(split_of)

# 5) Save processed dataset
out_path = processed_dir / "rul_features.parquet"
try:
    joined.to_parquet(out_path, index=False)
    print(f"\n✅ Saved: {out_path}")
    print(f"   Rows: {len(joined):,}")
    print(f"   Batteries: {len(unique_batts)}")
    print(f"   Train: {len(joined[joined['split']=='train']):,} rows")
    print(f"   Val: {len(joined[joined['split']=='val']):,} rows")
    print(f"   Test: {len(joined[joined['split']=='test']):,} rows")
except Exception as e:
    # Fallback to CSV if parquet fails
    out_path_csv = processed_dir / "rul_features.csv"
    joined.to_csv(out_path_csv, index=False)
    print(f"\n⚠️ Parquet save failed, saved as CSV instead: {out_path_csv}")
    print(f"   Error: {e}")
    print(f"   Rows: {len(joined):,}")


Extracting features from 2750 files...
  Processed 0/2750 files...
  Processed 100/2750 files...
  Processed 200/2750 files...
  Processed 300/2750 files...
  Processed 400/2750 files...
  Processed 500/2750 files...
  Processed 600/2750 files...
  Processed 700/2750 files...
  Processed 800/2750 files...
  Processed 900/2750 files...
  Processed 1000/2750 files...
  Processed 1100/2750 files...
  Processed 1200/2750 files...
  Processed 1300/2750 files...
  Processed 1400/2750 files...
  Processed 1500/2750 files...
  Processed 1600/2750 files...
  Processed 1700/2750 files...
  Processed 1800/2750 files...
  Processed 1900/2750 files...
  Processed 2000/2750 files...
  Processed 2100/2750 files...
  Processed 2200/2750 files...
  Processed 2300/2750 files...
  Processed 2400/2750 files...
  Processed 2500/2750 files...
  Processed 2600/2750 files...
  Processed 2700/2750 files...
Creating features DataFrame...
Successfully extracted features from 2750/2750 files
Joined dataset: 2750 

In [6]:
# Load and explore the processed dataset
processed_file = processed_dir / "rul_features.csv"
df = pd.read_csv(processed_file)

print("📊 Dataset Overview:")
print(f"   Total rows: {len(df):,}")
print(f"   Unique batteries: {df['battery_id'].nunique()}")
print(f"   Columns: {len(df.columns)}")
print(f"\n📋 Column names:")
print(df.columns.tolist())

print(f"\n📈 Dataset splits:")
print(df['split'].value_counts().sort_index())

print(f"\n📊 RUL Statistics:")
print(df['RUL'].describe())

print(f"\n🔋 Battery-level stats:")
batt_stats = df.groupby('battery_id').agg({
    'cycle_index': 'max',
    'RUL': ['min', 'max'],
    'SOH': ['min', 'max'],
    'Capacity': ['min', 'max']
}).round(3)
print(batt_stats.head(10))

print(f"\n✅ Dataset ready for modeling!")
print(f"   Feature columns (excluding metadata): {[c for c in df.columns if c not in ['battery_id', 'filename', 'type', 'start_time', 'test_id', 'uid', 'split', 'cycle_index', 'EOL_cycle', 'RUL', 'SOH', 'Capacity', 'Re', 'Rct', 'ambient_temperature']]}")


📊 Dataset Overview:
   Total rows: 2,750
   Unique batteries: 34
   Columns: 23

📋 Column names:
['type', 'start_time', 'ambient_temperature', 'battery_id', 'test_id', 'uid', 'filename', 'Capacity', 'Re', 'Rct', 'cycle_index', 'SOH', 'EOL_cycle', 'RUL', 'duration_s', 'voltage_mean', 'voltage_min', 'voltage_max', 'temp_max', 'current_mean_abs', 'coulomb_Ah', 'ir_drop_proxy', 'split']

📈 Dataset splits:
split
test      589
train    1735
val       426
Name: count, dtype: int64

📊 RUL Statistics:
count    1408.000000
mean       -3.636364
std        42.430040
min      -107.000000
25%       -35.000000
50%        -7.000000
75%        22.000000
max       123.000000
Name: RUL, dtype: float64

🔋 Battery-level stats:
           cycle_index    RUL           SOH        Capacity       
                   max    min    max    min    max      min    max
battery_id                                                        
B0005              168  -67.0  100.0  0.693  1.000    1.287  1.856
B0006           

In [7]:
# Convert CSV to Parquet for faster loading (now that pyarrow is installed)
csv_file = processed_dir / "rul_features.csv"
parquet_file = processed_dir / "rul_features.parquet"

if csv_file.exists():
    print("Converting CSV to Parquet format...")
    df_parquet = pd.read_csv(csv_file)
    df_parquet.to_parquet(parquet_file, index=False)
    print(f"✅ Saved parquet file: {parquet_file}")
    print(f"   CSV size: {csv_file.stat().st_size / (1024*1024):.2f} MB")
    print(f"   Parquet size: {parquet_file.stat().st_size / (1024*1024):.2f} MB")
    print(f"\n💡 Tip: Use pd.read_parquet('{parquet_file}') for faster loading in future notebooks!")
else:
    print("CSV file not found. Skipping conversion.")


Converting CSV to Parquet format...


ArrowKeyError: No type extension with name arrow.py_extension_type found

In [10]:
# Test EMD Feature Extraction
import sys
from pathlib import Path

# Add src to path
sys.path.append(str(Path("/Users/siddhantaggarwal/Desktop/Battery_RUL").resolve()))

from src.features import extract_all_features, extract_features_from_file
import pandas as pd

# Test on a single cycle file
test_file = root / "data" / "00001.csv"
print(f"Testing EMD extraction on: {test_file.name}")

cycle_data = pd.read_csv(test_file)
print(f"\nCycle data shape: {cycle_data.shape}")
print(f"Columns: {cycle_data.columns.tolist()}")

# Extract features with EMD
features = extract_all_features(cycle_data, include_emd=True, max_imfs=5)

print(f"\n✅ Extracted {len(features)} features")
print(f"\nFeature categories:")
feature_groups = {}
for key in features.keys():
    if '_imf' in key:
        group = 'EMD Features'
    elif key.startswith(('voltage', 'current', 'temp', 'duration', 'coulomb', 'ir')):
        group = 'Statistical Features'
    else:
        group = 'Other'
    
    if group not in feature_groups:
        feature_groups[group] = []
    feature_groups[group].append(key)

for group, feats in feature_groups.items():
    print(f"\n{group} ({len(feats)} features):")
    print(f"  Examples: {', '.join(feats[:5])}")
    if len(feats) > 5:
        print(f"  ... and {len(feats) - 5} more")


Testing EMD extraction on: 00001.csv

Cycle data shape: (490, 6)
Columns: ['Voltage_measured', 'Current_measured', 'Temperature_measured', 'Current_load', 'Voltage_load', 'Time']

✅ Extracted 16 features

Feature categories:

Statistical Features (16 features):
  Examples: duration_s, voltage_mean, voltage_min, voltage_max, voltage_std
  ... and 11 more
